<a href="https://colab.research.google.com/github/marcelAnd02/MachineLearning/blob/master/Text_gen_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import re
import random
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
raw_data = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/NLP_Project/PoetryFoundationData.csv')
dataset = raw_data.Poem.str.lower().values.tolist()

In [ ]:
#print(dataset[35])

In [ ]:
dataset = list(map(lambda x: re.findall(f"[\w]+|[.,'?!{chr(8217)}]", x), dataset))

In [ ]:
num_unique = 2000 #max number of unique words in model's vocab
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
tokenizer.fit_on_texts(dataset)
dataset_tokenized = tokenizer.texts_to_sequences(dataset)

In [ ]:
chain_len = 6
X = []
def filter_dataset():
  for ind, ex in enumerate(dataset_tokenized):
    for beg in range(0, len(ex) - chain_len):
      if np.max(ex[beg : beg + chain_len]) <= num_unique:
        X.append(dataset[ind][beg : beg + chain_len])

In [ ]:
filter_dataset()

In [ ]:
len(X)

1305661

In [ ]:
input_lng = 100000
#random.shuffle(X)
X = X[0 + 10000 : input_lng + 10000]
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [ ]:
X = np.asarray(X)
X = keras.utils.to_categorical(X)

In [ ]:
print(X.shape)
num_unique = X.shape[2]

(100000, 6, 1999)


In [ ]:
#tokenizer.sequences_to_texts([[127]])[0]

## AutoEncoder

In [ ]:
compact_size = 131072

In [ ]:
encoder = keras.models.Sequential(
    [
     keras.layers.LSTM(128, input_shape = (chain_len, num_unique), return_sequences = True),
     keras.layers.LSTM(128, return_sequences = True),
     keras.layers.LSTM(128, return_sequences = False),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(compact_size, activation = "selu"),
    ]
)
decoder = keras.models.Sequential(
    [
     keras.layers.Dense(512, activation = "selu", input_shape = [compact_size]),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.RepeatVector(chain_len),
     keras.layers.LSTM(256, return_sequences = True),
     keras.layers.LSTM(256, return_sequences = True),
     keras.layers.LSTM(256, return_sequences = True),
     keras.layers.TimeDistributed(keras.layers.Dense(num_unique)),
    ]
)
AutoEncoder = keras.models.Sequential([encoder, decoder])
optimizer = keras.optimizers.Adam(learning_rate = 0.0009)
AutoEncoder.compile(optimizer = optimizer, loss = "mse")

In [ ]:
AutoEncoder.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 131072)            35202816  
_________________________________________________________________
sequential_4 (Sequential)    (None, 6, 1999)           69461967  
Total params: 104,664,783
Trainable params: 104,664,783
Non-trainable params: 0
_________________________________________________________________


In [ ]:
AutoEncoder.fit(X, X, batch_size = 128, epochs = 140)

In [ ]:
#saving model
AutoEncoder.save('/content/drive/My Drive/Colab_Notebooks/NLP_Project/model5.h5')

In [ ]:
#loading saved model
AutoEncoder = keras.models.load_model('/content/drive/My Drive/Colab_Notebooks/NLP_Project/model5.h5')
encoder, decoder = AutoEncoder.layers

In [ ]:
def build_text(vec):
  pred = []
  for word in vec:
    pos = np.argmax(word)
    if pos == 0:
      continue
    pred.append(tokenizer.sequences_to_texts([[pos]])[0])
  return pred

Testing autoencoder:

In [ ]:
Xc = X[0].reshape((1, X[0].shape[0], X[0].shape[1]))
print(AutoEncoder.predict(Xc)[0].shape)

(6, 1999)


In [ ]:
ind = 9877
pred = AutoEncoder.predict(X[ind].reshape((1, X[ind].shape[0], X[ind].shape[1])))[0]
print(build_text(pred))
print(build_text(X[ind]))

['because', 'a', 'woman', "'", 's', '.']
['because', 'a', 'woman', "'", 's', '.']


In [ ]:
latent_vec = encoder.predict(X[ind].reshape((1, X[ind].shape[0], X[ind].shape[1])))
pred = decoder.predict(latent_vec)[0]
print(build_text(pred))
#print(latent_vec.shape)

['because', 'a', 'woman', "'", 's', '.']


## GAN

In [ ]:
noise_size = 20
generator = keras.models.Sequential(
    [
     keras.layers.Dense(512, activation = "selu", input_shape = [noise_size]),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(compact_size, activation = "selu"),
    ]
)
discriminator = keras.models.Sequential(
    [
     keras.layers.Dense(512, activation = "selu", input_shape = [compact_size]),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(256, activation = "selu"),
     keras.layers.Dense(1, activation = "sigmoid"),
    ]
)
gan = keras.models.Sequential([generator, discriminator])

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.0007)
discriminator.compile(optimizer = optimizer, loss = "binary_crossentropy")
discriminator.trainable = False
gan.compile(optimizer = optimizer, loss = "binary_crossentropy")

In [ ]:
batch_size = 256
def train_gan(num_epochs):
  generator, discriminator = gan.layers
  discriminator.trainable = False
  for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1} / {num_epochs}")
    for i in range(0, X.shape[0] - batch_size, batch_size):
      #discrminator training
      noise = tf.random.normal(shape = [batch_size, noise_size])
      gen_sentences = generator(noise)
      encoded_input = encoder(X[i : i + batch_size])
      X_curr = np.concatenate((encoded_input, gen_sentences), axis = 0)
      Y1 = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))))
      discriminator.trainable = True
      discriminator.train_on_batch(X_curr, Y1)
      #generator training
      noise = tf.random.normal(shape = [batch_size, noise_size])
      Y2 = np.ones((batch_size, 1))
      discriminator.trainable = False
      gan.train_on_batch(noise, Y2)

In [ ]:
train_gan(90)

Testing GAN:

In [ ]:
noise = tf.random.normal(shape = [1, noise_size])
gen_vec = generator(noise)
encoded_vec = decoder(gen_vec)
build_text(encoded_vec[0])

['still', 'to', 'say', 'other', 'about', 'her']